In [1]:
!which python

/home/ec2-user/anaconda3/envs/python3/bin/python


In [2]:
!pip install cmake==3.27.0 pyarrow==16.1.0 datasets transformers accelerate sagemaker boto3 --upgrade --no-cache-dir

INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 416.0 MB/s  0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.40.69
    Uninstalling botocore-1.40.69:
      Successfully uninstalled botocore-1.40.69━ 0/2 [botocore]
  Attempting uninstall: boto3━━━━━━━━━━━━━━━━━━━ 0/2 [botocore]
    Found existing installation: boto3 1.40.690m 0/2 [botocore]
    Uninstalling boto3-1.40.69:━━━━━━━━━━━━━ 0/2 [botocore]
      Successfully uninstalled boto3-1.40.69 0/2 [botocore]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [boto3]32m1/2 [boto3]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.42.69 requires botocore==1.40.69, but you have botocore 1.40.74 which is incompatible.


In [3]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 30.0 MB/s  0:00:00


In [4]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 194.6 MB/s  0:00:00 eta 0:00:01


In [8]:
import boto3

In [9]:
s3=boto3.client("s3")

In [10]:
response = s3.list_objects_v2(Bucket='llm-fine-tune-dataset-mrv',Prefix='dataset/')

In [11]:
for obj in response.get("Contents", []):
    print(obj)

{'Key': 'dataset/', 'LastModified': datetime.datetime(2025, 11, 11, 15, 48, 52, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 0, 'StorageClass': 'STANDARD'}
{'Key': 'dataset/pharma_instruction_data.csv', 'LastModified': datetime.datetime(2025, 11, 11, 19, 43, 52, tzinfo=tzlocal()), 'ETag': '"7a3bbfb7acae4d8dc88dc1a08501d745"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 2469, 'StorageClass': 'STANDARD'}


In [12]:
for obj in response.get("Contents", []):
    print(obj["Key"])

dataset/
dataset/pharma_instruction_data.csv


In [16]:
dataset_path = 's3://llm-fine-tune-dataset-mrv/dataset/pharma_instruction_data.csv'

In [17]:
from datasets import load_dataset

In [18]:
dataset = load_dataset("csv",data_files={"train":dataset_path},split="train")
# dataset = load_dataset("csv", data_files="/content/pharma_instruction_data.csv",split="train")

In [19]:
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5
})

In [20]:
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5
})


In [21]:
print(dataset[0])

{'instruction': 'Explain the mechanism of action of Metformin.', 'input': None, 'output': 'Metformin activates AMP-activated protein kinase (AMPK), which increases glucose uptake and fatty-acid oxidation while inhibiting hepatic gluconeogenesis, thereby lowering blood glucose.'}


In [22]:
def format_example(example):
    prompt = f"### Instruction:\n{example['instruction']}\n### Input:\n{example['input']}\n### Response:\n{example['output']}"
    return {"text": prompt}

In [23]:
dataset = dataset.map(format_example)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [24]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 5
})

In [54]:
dataset['text'][0]

'### Instruction:\nExplain the mechanism of action of Metformin.\n### Input:\nNone\n### Response:\nMetformin activates AMP-activated protein kinase (AMPK), which increases glucose uptake and fatty-acid oxidation while inhibiting hepatic gluconeogenesis, thereby lowering blood glucose.'

In [55]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

In [56]:
model_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

In [58]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [59]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [60]:
def tokenize_fn(example):
    tokens = tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [69]:
tokenized = dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [71]:
tokenized['input_ids']

Column([[1, 835, 2799, 4080, 29901, 13, 9544, 7420, 278, 13336, 310, 3158, 310, 4737, 689, 262, 29889, 13, 2277, 29937, 10567, 29901, 13, 8516, 13, 2277, 29937, 13291, 29901, 13, 10095, 689, 262, 5039, 1078, 319, 3580, 29899, 11236, 630, 26823, 19015, 559, 313, 19297, 29968, 511, 607, 16415, 3144, 1682, 852, 318, 415, 1296, 322, 9950, 1017, 29899, 562, 333, 19100, 333, 362, 1550, 297, 6335, 11407, 540, 29886, 2454, 3144, 29884, 535, 29872, 6352, 6656, 29892, 27999, 5224, 292, 10416, 3144, 1682, 852, 29889, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [68]:
from peft import LoraConfig, get_peft_model, TaskType

In [65]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

RuntimeError: unable to mmap 4400216536 bytes from file </home/ec2-user/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-intermediate-step-1431k-3T/snapshots/59f6f375b26bde864a6ca194a9a3044570490064/model.safetensors>: Cannot allocate memory (12)

In [70]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none"
)

In [67]:
model = get_peft_model(model, lora_config)

NameError: name 'model' is not defined

In [45]:
args = TrainingArguments(
    output_dir="./tinyllama-instruction",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_total_limit=1,
    report_to="none"
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [46]:

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
)

NameError: name 'model' is not defined

In [47]:
trainer.train()

NameError: name 'trainer' is not defined

In [48]:

model_path = "/content/tinyllama-instruction/checkpoint-3"

In [49]:

instruction_model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/tinyllama-instruction/checkpoint-3'. Use `repo_type` argument if needed.

In [50]:

prompt = "Explain the mechanism of action of Metformin."

In [51]:

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

NameError: name 'tokenizer' is not defined

In [52]:

outputs = instruction_model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.8,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.1
)

NameError: name 'instruction_model' is not defined

In [53]:

print("\nModel Output:\n")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Model Output:



NameError: name 'tokenizer' is not defined